# Programming task — MDS

In [41]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Importing data

In [23]:
ep6 = pd.read_excel(
    'VoteWatch-EP-voting-data_2004-2022/EP6_RCVs_2022_06_13.xlsx', header=1)
ep7 = pd.read_excel(
    'VoteWatch-EP-voting-data_2004-2022/EP7_RCVs_2014_06_19.xlsx', header=0)
ep8 = pd.read_excel(
    'VoteWatch-EP-voting-data_2004-2022/EP8_RCVs_2019_06_25.xlsx', header=0)
ep9 = pd.read_excel(
    'VoteWatch-EP-voting-data_2004-2022/EP9_RCVs_2022_06_22.xlsx', header=0)


In [28]:
ep6_voted = pd.read_excel(
    'VoteWatch-EP-voting-data_2004-2022/EP6_Voted docs.xlsx', header=0)
ep7_voted = pd.read_excel(
    'VoteWatch-EP-voting-data_2004-2022/EP7_Voted docs.xlsx', header=0)
ep8_voted = pd.read_excel(
    'VoteWatch-EP-voting-data_2004-2022/EP8_Voted docs.xlsx', header=0)
ep9_voted = pd.read_excel(
    'VoteWatch-EP-voting-data_2004-2022/EP9_Voted docs.xlsx', header=0)

In [29]:
votes = pd.read_excel(
    'EU-Council-votes-July-2009-to-Feb-2022.xlsx', header=0)

In [30]:
rcv_legend = ['not a MEP', 'for', 'against',
              'abstention', 'absent', 'didn\'t vote', 'motivated']

In [32]:
ep6.drop(index=0, inplace=True, errors='ignore')

## Pre-processing

In [33]:
countries = ep6['Country'].unique()
countries

array(['Cyprus', 'Poland', 'Italy', 'United Kingdom', 'Germany', 'Sweden',
       'Latvia', 'Lithuania', 'France', 'Greece', 'Portugal', 'Malta',
       'Denmark', 'Spain', 'Ireland', 'Slovakia', 'Hungary',
       'Netherlands', 'Belgium', 'Austria', 'Czech Republic', 'Slovenia',
       'Luxembourg', 'Finland', 'Estonia', 'Bulgaria', 'Romania'],
      dtype=object)

In [34]:
epgs = ep6['EPG'].unique()
epgs

array(['Confederal Group of the European United Left - Nordic Green Left',
       'Independence/Democracy Group',
       "Group of the European People's Party (Christian Democrats) and European Democrats",
       'Non-attached Members',
       'Group of the Alliance of Liberals and Democrats for Europe',
       'Socialist Group in the European Parliament',
       'Union for Europe of the Nations Group',
       'Group of the Greens/European Free Alliance'], dtype=object)

In [55]:
def get_first_vot_col(df):
    """
    Returns the index of the first column containing voting data
    
    Parameters
    ----------
    df : pandas.DataFrame
        A dataframe containing voting data
    """
    return np.where(df.columns == '1')[0][0]

In [62]:
def vote_sim(df, i, j):
    """
    Returns the percentage of votes that two MEPs have in common

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the voting data
    i : int
        The row index of the first MEP
    j : int
        The row index of the second MEP
    """
    return np.mean(df.iloc[i, get_first_vot_col(df):len(df.columns)] == df.iloc[j, get_first_vot_col(df):len(df.columns)])

1       5.0
2       5.0
3       3.0
4       3.0
5       3.0
       ... 
6196    0.0
6197    0.0
6198    0.0
6199    0.0
6200    0.0
Name: 2, Length: 6199, dtype: object